## Import packages

In [ ]:
! pip install transformers demoji vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 125 kB 72.2 MB/s 
     |████████████████████████████████| 6.6 MB 65.8 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import os
import re
import string
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
import demoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import tensorflow as tf
from sklearn.model_selection import train_test_split
import transformers
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification, TFBertForSequenceClassification

In [ ]:
demoji.download_codes()
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  """Entry point for launching an IPython kernel.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/Colab Notebooks/MSDS498/')

Mounted at /content/drive


## Functions

In [ ]:
#convert emojis to text
def remove_em(text):

  dem = demoji.findall(text)

  for item in dem.keys():
    text = text.replace(item, dem[item])

  return text

In [ ]:
def sentiment_scores(sentence):
	# Create a SentimentIntensityAnalyzer object.
  sid_obj = SentimentIntensityAnalyzer()

	# polarity_scores method of SentimentIntensityAnalyzer
	# object gives a sentiment dictionary.
	# which contains pos, neg, neu, and compound scores.
  sentiment_dict = sid_obj.polarity_scores(sentence)
	
	#print("Overall sentiment dictionary is : ", sentiment_dict)
	#print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
	#print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
	#print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")

  #print("Sentence Overall Rated As", end = " ")

	# decide sentiment as positive, negative and neutral
  if sentiment_dict['compound'] >= 0.05:
    sent = 'Positive'
  elif sentiment_dict['compound'] <= - 0.05:
    sent = 'Negative'
  else :
    sent = 'Neutral'
  #print(sent)
  return sent

In [ ]:
def predict_proba(text_list):
  
  encodings = tokenizer(text_list, max_length=MAX_LEN, truncation=True, padding=True)
  dataset = tf.data.Dataset.from_tensor_slices((dict(encodings))) 
  preds = model.predict(dataset.batch(1)).logits
  res = tf.nn.softmax(preds, axis=1).numpy()
  pred1 = np.argmax(res, axis=1)
  
  return pred1

## Import data

In [ ]:
df = pd.read_csv('df_twt.csv', lineterminator='\n')

In [ ]:
df.head()

,created_at,id,text
0,2022-04-21T23:59:46.000Z,1517292038324162560,Twitter's Thoughts:\n\n$UAL - Very Positive \n...
1,2022-04-21T23:59:04.000Z,1517291863430082560,$GPRO $NFLX $PTON NEW ARTICLE : Peloton: Netfl...
2,2022-04-21T23:57:16.000Z,1517291408100581376,JOIN ME TONIGHT $OCGN $ATER $NILE $SPY $QQQ $N...
3,2022-04-21T23:57:06.000Z,1517291366056869888,$SPX is this start of rally 📉 towards $3800-$3...
4,2022-04-21T23:56:58.000Z,1517291332011585536,$NFLX 100$ stock ⭐


In [ ]:
for i in range(10):
  print(df['text'][i])

Twitter's Thoughts:

$UAL - Very Positive 
$IIIN - Very Positive 
$SPY - Very Positive 
$CDVA.C - Very Positive 
$NFLX - Very Positive 

#stocks #investing #trading
$GPRO $NFLX $PTON NEW ARTICLE : Peloton: Netflix's Earnings Plunge Is A Big Warning https://t.co/bWjAywh3sv Get all the latest $GPRO related news here : https://t.co/iWRqecupTV
JOIN ME TONIGHT $OCGN $ATER $NILE $SPY $QQQ $NFLX $SNAP $CRXT #StockMarket #fintwit #stocks https://t.co/Z4t9xUtKa2
$SPX is this start of rally 📉 towards $3800-$3900 ??
$FB &amp; $NFLX down over 50% 
Growth stocks down over 80% 
$AAPL $AMZN $GOOGL $NVDA next in line!!

$TSLA will have its day, this can see dip to max $600 if MM wants to drain it. “Stock Split” is damage control. https://t.co/MwJW22mGrH
$NFLX 100$ stock ⭐
$NFLX they will go bankrupt over this imo ️
🔥 4/21  recap : 🔥

$Roku p  0.71 - went 3.51$
$Nflx p  1.08 went 7.80$
$Fb p  2.10  went 4.00 $

$Spy p round 1 50%  ( went 200%
$Spy p round 2 44 % ( went 150% )

$Tsla c 12%- ❌️

ER Trade

In [ ]:
df0 = pd.read_csv('df_twt.csv', lineterminator='\n')
df1 = pd.read_csv('df_twt_1.csv', lineterminator='\n')
df2 = pd.read_csv('df_twt_2.csv', lineterminator='\n')
df3 = pd.read_csv('df_twt_3.csv', lineterminator='\n')

In [ ]:
df = pd.concat([df0, df1, df2, df3])
df = df.drop('withheld', axis=1)
df.shape

(313356, 3)

In [ ]:
df['text'].isna().sum()

222

In [ ]:
#some data was transferred over in the wrong columns -- without the created at dates, the data is not that useful, so it will be dropped (could fill in later with the nearest dates in earlier columns)
df[df.isna().any(axis=1)]

,created_at,id,text
20703,Via https://t.co/1eBkr88tlf,NaN,NaN
20704,#nflx #wallstreetbets https://t.co/92p5HMuZi5,NaN,NaN
21643,Via https://t.co/1eBkr88tlf,NaN,NaN
21644,#nflx #wallstreetbets https://t.co/xShrQHd98q,NaN,NaN
22880,https://t.co/f5pQyfdDFB,NaN,NaN
...,...,...,...
38987,$DE 292.95 to 285.50,NaN,NaN
38988,$NFLX 541.00 to 530.33,NaN,NaN
38989,$MSFT 238.02 to 231.35,NaN,NaN
38990,$INTU 369.89 to 359.69,NaN,NaN


In [ ]:
df = df.dropna()
df.shape

(313134, 3)

## Preprocess tweets

In [ ]:
df['text'] = df['text'].apply(remove_em) #convert emojis with words

In [ ]:
df.to_csv('df_full.csv', index=False)

## Experiment 1

### Create a subset

In [ ]:
# Randomly sample 40000 of dataframe
df_subset = df.sample(n = 40000)

In [ ]:
df_subset['no_processing_sentiment'] = df_subset['text'].apply(sentiment_scores) 

In [ ]:
df_subset.to_csv('df_subset_40000.csv', index=False)

In [ ]:
#replace emojis with words
#df['text'] = df['text'].apply(remove_em)
#remove URLs
df_subset.process_text = df_subset.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
df_subset.process_text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
#remove usernames (@)
df_subset.process_text = df_subset.process_text.apply(lambda x: re.sub('@[^\s]+', '', x))
#remove hashtags
df_subset.process_text = df_subset.process_text.apply(lambda x: re.sub('#[^\s]+', '', x))
#remove other stock mentions
df_subset.process_text = df_subset.process_text.apply(lambda x: re.sub('\$[a-zA-Z]+', '', x))
#remove new line \n 
df_subset.process_text = df_subset.process_text.apply(lambda x: re.sub('\\n', '', x))
#remove dashes 
df_subset.process_text = df_subset.process_text.apply(lambda x: re.sub('\-', '', x))
#remove numbers
df_subset.process_text = df_subset.process_text.apply(lambda x: re.sub('[0-9]+', '', x))
#remove symbols
df_subset.process_text = df_subset.process_text.apply(lambda x: re.sub('[^\w\s]', '', x))
#remove stopwords
df_subset.process_text = df_subset.process_text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords.words('english'))]))
#remove words with 2 or less characters
df_subset.process_text = df_subset.process_text.apply(lambda x: ' '.join([word for word in x.split() if len(word)>2]))
#lowercase - VADER takes into account CAPS
#df["text"] = df["text"].str.lower()

In [ ]:
df_subset['yes_processing_sentiment'] = df_subset.process_text.apply(sentiment_scores) 

In [ ]:
df_subset_diff = df_subset[df_subset['no_processing_sentiment'] !=df_subset['yes_processing_sentiment']]

In [ ]:
df_subset_diff = df_subset_diff[['text','no_processing_sentiment','yes_processing_sentiment']]
df_subset_diff

,text,no_processing_sentiment,yes_processing_sentiment
20128,$NFLX is down -37.5% 🆓,Positive,Neutral
55671,$AAPL $TSLA $AMZN $FB $NFLX $AMD $SPY\n$MSFT $...,Positive,Negative
53482,Did you see the flow on $NFLX today?😍😍😍 https:...,Positive,Neutral
38824,$HD $LOW $NFLX $SQ $TSLA $AMC\n\nToday's Watch...,Negative,Neutral
87641,@breadcrumbsre Joey Levin: $IAC \nAnjali Sud (...,Positive,Neutral
...,...,...,...
11481,Woaaa which shitcoin chart is this? #SAFEMOON ...,Negative,Neutral
62421,"$INTC moving as discussed, I'm in all my accou...",Positive,Neutral
45158,#MFAAAANNGT \n$MSFT $FB $AMZN $APPL $AMD $NVDA...,Negative,Positive
47374,$IWM 10Sep $227p limit buy $1.55🔥🔥 markets loo...,Negative,Neutral


There does seem to be some difference in sentiment based on preprocessing the data with VADER.

In [ ]:
sample_eval = df_subset_diff.sample(n = 30)

In [ ]:
for i in range(30):
  print(sample_eval['text'].iloc[i])
  print(sample_eval.iloc[i])
  print('\n')

$NFLX right down to the Gold target area and bounced....reversal time ???  Lower time frames diverging.  :)
text                        $NFLX right down to the Gold target area and b...
no_processing_sentiment                                              Positive
yes_processing_sentiment                                             Negative
Name: 499, dtype: object


Excellent companies that you can invest in are all around you!

☕️Want a beverage? $SBUX $MNST $STZ
🍟Want food? $MCD $CMG $WEN
🛍You going grocery shopping? $WMT $TGT $COST
📺Watching a movie? $DIS $NFLX $ROKU
🏘Fixing the house? $HD $LOW

Buy what you know! 🔥
text                        Excellent companies that you can invest in are...
no_processing_sentiment                                              Negative
yes_processing_sentiment                                             Positive
Name: 29182, dtype: object


$NFLX 😂🤣 https://t.co/ivH2GjwLKL
text                        $NFLX 😂🤣 https://t.co/ivH2GjwLKL
no_processing_se

Reading through some tweets, there are a lot of ads that convey positive sentiment but does not talk about the stock itself. Some tweets mention the stock but to tag it to attract more attention, not actually about the stock. VADER doesn't seem to be the most accurate of tools. Lots of tweets seem to be more neutral rather than positive or negative.

From looking over a subset of the data, it seems that the no_processing_sentiment seems to be a more accurate representation than when the data is processed. The no_processing_sentiment wil be used to fine-tune the BERT models.

In [ ]:
df_subset = df_subset[['text','no_processing_sentiment']]
df_subset.head()

,text,no_processing_sentiment
65093,$NFLX was a great business when it was purely ...,Positive
70640,$NFLX https://t.co/gC7oO452o2,Neutral
79905,$nflx upgrade. All day runner. 540 560 next ...,Neutral
20128,$NFLX is down -37.5% 🆓,Positive
58624,Having a trusted team on your side will make a...,Positive


In [ ]:
df_subset.groupby('no_processing_sentiment').count()

,text
no_processing_sentiment,
Negative,1855
Neutral,3346
Positive,4799


The data is not balanced, so I will instead just take 1855 of each of the sentiments to use for my fine-tuning data set. 

In [ ]:
pos = df_subset[df_subset['no_processing_sentiment']=='Positive']
neu = df_subset[df_subset['no_processing_sentiment']=='Neutral']
neg = df_subset[df_subset['no_processing_sentiment']=='Negative']

In [ ]:
pos = pos.sample(n = 1855)
neu = neu.sample(n = 1855)

In [ ]:
df_finetune = pd.concat([pos,neu,neg])
df_finetune.head()

,text,no_processing_sentiment
34121,Mid-day MACD bearish X-Over with greatest % be...,Positive
3343,Could Netflix Help Shares Of Formula One? $NFL...,Positive
89223,Best discord group over 80k members and even h...,Positive
51785,SweepCast alerted: $NFLX with Unusual Options ...,Positive
82599,"Bye Silicon Valley, investors are leaving… Tim...",Positive


In [ ]:
df_finetune.to_csv('df_finetune.csv', index=False)

In [ ]:
df_finetune.loc[df_finetune['no_processing_sentiment'] == 'Positive', 'no_processing_sentiment'] = 0
df_finetune.loc[df_finetune['no_processing_sentiment'] == 'Neutral', 'no_processing_sentiment'] = 1
df_finetune.loc[df_finetune['no_processing_sentiment'] == 'Negative', 'no_processing_sentiment'] = 2

In [ ]:
#remove URLs
df_finetune.text = df_finetune.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
df_finetune.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
#remove new line \n 
df_finetune.text = df_finetune.text.apply(lambda x: re.sub('\\n', ' ', x))
#remove symbols
df_finetune.text = df_finetune.text.apply(lambda x: re.sub('[^\w\s]', '', x))

In [ ]:
df_finetune.groupby('no_processing_sentiment').count()

,text
no_processing_sentiment,
0,1855
1,1855
2,1855


### Fine-tune FinBert

In [ ]:
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369 [00:00<?, ?B/s]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_finetune['text'], df_finetune['no_processing_sentiment'], test_size = 0.20, random_state = 42)

In [ ]:
MAX_LEN = X_train.apply(lambda s: len([x for x in s.split()])).max()
MAX_LEN

60

In [ ]:
train_encodings = tokenizer(list(X_train.astype(str)),
                            truncation=True, 
                            padding=True)
test_encodings = tokenizer(list(X_test.astype(str)),
                           truncation=True, 
                           padding=True)

In [ ]:
y_train_array = np.asarray(y_train).astype('float32')
y_test_array = np.asarray(y_test).astype('float32')

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train_array
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test_array
))

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis', num_labels=3, from_pt=True)

Downloading:   0%|          | 0.00/789 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# train the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(8),
          epochs=5,
          batch_size=16)

Epoch 1/5
557/557 [==============================] - 127s 183ms/step - loss: 10.7239 - accuracy: 0.3347
Epoch 2/5
557/557 [==============================] - 102s 183ms/step - loss: 10.7092 - accuracy: 0.3356
Epoch 3/5
557/557 [==============================] - 102s 183ms/step - loss: 10.7092 - accuracy: 0.3356
Epoch 4/5
557/557 [==============================] - 102s 183ms/step - loss: 10.7092 - accuracy: 0.3356
Epoch 5/5
557/557 [==============================] - 108s 193ms/step - loss: 10.7092 - accuracy: 0.3356


In [ ]:
tf.keras.models.save_model(model,'/content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment1',save_format="tf")

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment1/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment1/assets


In [ ]:
loaded = tf.saved_model.load('/content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment1')

In [ ]:
# model evaluation on the test set
model.evaluate(test_dataset.shuffle(len(X_test)).batch(8), return_dict=True, batch_size=16)

140/140 [==============================] - 21s 117ms/step - loss: 10.8902 - accuracy: 0.3243


{'accuracy': 0.3243485987186432, 'loss': 10.890220642089844}

### Run tweets through fine-tuned model

In [ ]:
df_full = pd.read_csv("df_full.csv", lineterminator='\n') #full dataset with emojis converted to text
df_full.head()

#remove URLs
df_full.text = df_full.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
df_full.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
#remove new line \n 
df_full['text'] = df_full['text'].apply(lambda x: re.sub('\\n', ' ', x))
#remove symbols
df_full['text'] = df_full['text'].apply(lambda x: re.sub('[^\w\s]', '', x))

In [ ]:
textList = df_full.text.tolist()

In [ ]:
sent = predict_proba(textList)

In [ ]:
df_2 = pd.DataFrame(sent)

In [ ]:
fin = pd.concat([df_full, df_2], axis= 1)

In [ ]:
fin.columns = ['date','id','text','fine-tuned_finbert_sentiment']

In [ ]:
fin.head()

,date,id,text,fine-tuned_finbert_sentiment
0,2022-04-21T23:59:46.000Z,1.517292e+18,Twitters Thoughts UAL Very Positive IIIN V...,2
1,2022-04-21T23:59:04.000Z,1.517292e+18,GPRO NFLX PTON NEW ARTICLE Peloton Netflixs E...,2
2,2022-04-21T23:57:16.000Z,1.517291e+18,JOIN ME TONIGHT OCGN ATER NILE SPY QQQ NFLX SN...,2
3,2022-04-21T23:57:06.000Z,1.517291e+18,SPX is this start of rally chart decreasing to...,2
4,2022-04-21T23:56:58.000Z,1.517291e+18,NFLX 100 stock star,2


In [ ]:
fin.to_csv('df_full_fine_tuned_experiment1.csv', index=False)

In [ ]:
fin = pd.read_csv('df_full_fine_tuned_experiment1.csv', lineterminator='\n')

In [ ]:
fin.groupby('fine-tuned_finbert_sentiment').count()

,date,id,text
fine-tuned_finbert_sentiment,,,
2,313134,313134,313134


## Experiment 2

### Create a subset

In [ ]:
df = pd.read_csv('df_full.csv', lineterminator='\n')

In [ ]:
df.shape

(313134, 3)

In [ ]:
# Randomly sample dataframe
df_subset = df.sample(n = 80000)

In [ ]:
df_subset['no_processing_sentiment'] = df_subset['text'].apply(sentiment_scores) 

In [ ]:
df_subset.to_csv('df_subset_80000.csv', index=False)

In [ ]:
df_subset = df_subset[['text','no_processing_sentiment']]
df_subset.head()

,text,no_processing_sentiment
286412,"$ALXN, $BXP, $CAR, $CHWY, $CME, $CWH, $CXO, $E...",Positive
18070,EOP can find trades ahead of the open and be d...,Neutral
20930,It will be an OTC Hall of Fame 2021.\nright ar...,Positive
179109,https://t.co/uFuxkWHorn Sports related stock t...,Neutral
295780,The stock market isn’t reflecting reality ... ...,Negative


In [ ]:
df_subset.groupby('no_processing_sentiment').count()

,text
no_processing_sentiment,
Negative,14269
Neutral,27196
Positive,38535


The data is not balanced, so I will instead just take 4000 of each of the sentiments to use for my fine-tuning data set. 

In [ ]:
pos = df_subset[df_subset['no_processing_sentiment']=='Positive']
neu = df_subset[df_subset['no_processing_sentiment']=='Neutral']
neg = df_subset[df_subset['no_processing_sentiment']=='Negative']

In [ ]:
pos = pos.sample(n = 4000)
neu = neu.sample(n = 4000)
neg = neg.sample(n = 4000)

In [ ]:
df_finetune = pd.concat([pos,neu,neg])
df_finetune.head()

,text,no_processing_sentiment
4934,@jimcramer yo Jimmy lol do you want us buy $NF...,Positive
124146,Great place to learn trading and to get instan...,Positive
199967,Alerts before spikes and right as big news dro...,Positive
232464,$NFLX new alert at https://t.co/A7qrDarJHY #...,Positive
299067,Read and understand... These Suckers have paid...,Positive


In [ ]:
df_finetune.to_csv('df_finetune_4000.csv', index=False)

In [ ]:
df_finetune = pd.read_csv('df_finetune_4000.csv', lineterminator='\n')

In [ ]:
df_finetune.loc[df_finetune['no_processing_sentiment'] == 'Positive', 'no_processing_sentiment'] = 0
df_finetune.loc[df_finetune['no_processing_sentiment'] == 'Neutral', 'no_processing_sentiment'] = 1
df_finetune.loc[df_finetune['no_processing_sentiment'] == 'Negative', 'no_processing_sentiment'] = 2

In [ ]:
#remove URLs
df_finetune.text = df_finetune.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
df_finetune.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
#remove new line \n 
df_finetune.text = df_finetune.text.apply(lambda x: re.sub('\\n', ' ', x))
#remove symbols
df_finetune.text = df_finetune.text.apply(lambda x: re.sub('[^\w\s]', '', x))

In [ ]:
df_finetune.groupby('no_processing_sentiment').count()

,text
no_processing_sentiment,
0,4000
1,4000
2,4000


### Fine-tune FinBert

In [ ]:
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369 [00:00<?, ?B/s]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_finetune['text'], df_finetune['no_processing_sentiment'], test_size = 0.20, random_state = 42)

In [ ]:
MAX_LEN = X_train.apply(lambda s: len([x for x in s.split()])).max()
MAX_LEN

86

In [ ]:
train_encodings = tokenizer(list(X_train.astype(str)),
                            truncation=True, 
                            padding=True)
test_encodings = tokenizer(list(X_test.astype(str)),
                           truncation=True, 
                           padding=True)

In [ ]:
y_train_array = np.asarray(y_train).astype('float32')
y_test_array = np.asarray(y_test).astype('float32')

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train_array
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test_array
))

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('ahmedrachid/FinancialBERT-Sentiment-Analysis', num_labels=3, from_pt=True)

Downloading:   0%|          | 0.00/789 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# train the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(8),
          epochs=3,
          batch_size=16)

Epoch 1/3
1200/1200 [==============================] - 216s 164ms/step - loss: 10.7065 - accuracy: 0.3301
Epoch 2/3
1200/1200 [==============================] - 196s 163ms/step - loss: 10.7924 - accuracy: 0.3304
Epoch 3/3
1200/1200 [==============================] - 196s 163ms/step - loss: 10.7924 - accuracy: 0.3304


In [ ]:
tf.keras.models.save_model(model,'/content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment2',save_format="tf")

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment2/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment2/assets


In [ ]:
loaded = tf.saved_model.load('/content/drive/My Drive/Colab Notebooks/MSDS498/Model/Experiment2')

In [ ]:
# model evaluation on the test set
model.evaluate(test_dataset.shuffle(len(X_test)).batch(8), return_dict=True, batch_size=16)

300/300 [==============================] - 23s 66ms/step - loss: 10.5574 - accuracy: 0.3450


{'accuracy': 0.3449999988079071, 'loss': 10.557355880737305}

### Run tweets through fine-tuned model

In [ ]:
df_full = pd.read_csv("df_full.csv", lineterminator='\n') #full dataset with emojis converted to text
df_full.head()

#remove URLs
df_full.text = df_full.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
df_full.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
#remove new line \n 
df_full['text'] = df_full['text'].apply(lambda x: re.sub('\\n', ' ', x))
#remove symbols
df_full['text'] = df_full['text'].apply(lambda x: re.sub('[^\w\s]', '', x))

NameError: ignored

In [ ]:
textList = df_full.text.tolist()

In [ ]:
sent = predict_proba(textList)

In [ ]:
df_2 = pd.DataFrame(sent)

In [ ]:
fin = pd.concat([df_full, df_2], axis= 1)

In [ ]:
fin.columns = ['date','id','text','fine-tuned_finbert_sentiment']

In [ ]:
fin.head()

In [ ]:
fin.to_csv('df_full_fine_tuned_experiment2.csv', index=False)

In [ ]:
fin.groupby('fine-tuned_finbert_sentiment').count()